In [1]:
import multiprocessing
from pathlib import Path
import pandas as pd

In [2]:
all_data = Path('./testdata/').glob('*.vcf')

In [3]:
all_path = []
for d in all_data:
    all_path.append(d)

In [4]:
# function to get array with chromsome, position, ref, alt, in single .vcf file
def getindex(file):
    array = []
    with file.open() as f:
        for line in f:
            if not line.startswith('#'):
                try:
                    block = line.split('\t')
                    chromosome = block[0]
                    position = block[1]
                    ref = block[3]
                    alt = block[4]
                    PK = chromosome + ':' + position + ':' + ref + ':' + alt

                    array.append(PK)
                except:
                    print(line + '---  error')
                
    return(array)

In [5]:
# function to append array from every vcf to unique set.
shareset = set()
def addtoset(result):
    for res in result:
        for r in res:
            shareset.add(r)

In [6]:
p = multiprocessing.Pool(4)

In [7]:
p.map_async(getindex, all_path, callback=addtoset)
p.close()
p.join()


---  error

---  error

---  error

---  error

---  error

---  error

---  error

---  error


In [8]:
shareset

{'chr1:88388:C:T',
 'chr1:361880:T:G',
 'chr1:61208:T:C',
 'chr1:537930:G:T',
 'chr1:584537:C:T',
 'chrM:6186:T:C',
 'chr1:782981:C:T',
 'chrM:1890:G:A',
 'chr1:242989:C:T',
 'chr1:28802:A:T',
 'chr1:123642:T:G',
 'chrM:5163:T:C',
 'chr1:69897:T:C',
 'chr1:160860:A:G',
 'chr1:39231:CCT:C',
 'chr1:618829:A:G',
 'chr1:126847:T:C',
 'chr1:167681:A:G',
 'chr1:846864:G:C',
 'chrM:7434:C:T',
 'chr1:443006:A:G',
 'chrM:16149:C:T',
 'chr1:140188:C:A',
 'chr1:743072:C:A',
 'chr1:162532:G:T',
 'chr1:88905:G:A',
 'chr1:593236:A:G',
 'chr1:79663:A:G',
 'chr1:125957:G:A',
 'chrM:14693:A:G',
 'chr1:119485:C:A',
 'chr1:711310:G:A',
 'chr1:787185:G:A',
 'chr1:80599:G:T',
 'chr1:564039:C:A',
 'chr1:552197:G:A',
 'chrM:9933:G:A',
 'chr1:118241:A:T',
 'chr1:753404:GC:G,GA',
 'chr1:10446:A:AC',
 'chr1:625199:C:T',
 'chr1:830181:A:G',
 'chr1:594036:A:T',
 'chr1:440438:G:A',
 'chr1:21224:G:A',
 'chr1:772306:T:G',
 'chr1:84006:G:A',
 'chr1:174556:T:C',
 'chr1:54773:TTCCTCCTTTTCTTTCC:T',
 'chr1:542699:T:C',
 

In [8]:
from sys import getsizeof
getsizeof(shareset)/1024/1024

0.500213623046875

In [10]:
df = pd.DataFrame(index=shareset)

In [21]:
shareset_list = []
for line in shareset:
    shareset_list.append(line)
shareset_list = sorted(shareset_list)

In [87]:
## map type
def getGenotype(single_vcf_file):
    vcf = single_vcf_file.open()

    PK_sample = dict()  # dict(vcf_per_line:index)
    Sample = []
    line_count = 0

    for num, line in enumerate(vcf):
        if line.startswith('#'):
            pass
        else:
            block = line.split('\t')
            chromosome = block[0]
            position = block[1]
            ref = block[3]
            alt = block[4]
            PK = chromosome + ":" + position + ":" +ref + ":" + alt
            PK_sample[PK] = line_count
            line_count += 1
            Sample.append(line)

    genotype = ['none']*len(shareset_list)
    for i in range(0, len(shareset_list)):
        per_unique_loc = shareset_list[i]

        # get the vcf line of sample, and transfer the GT label.
        if per_unique_loc in PK_sample.keys():
            index = PK_sample[per_unique_loc]
            block = Sample[index].split('\t')
            ref = block[3]
            alt = block[4].split(',')
            if alt != '<CNV>':
                table = [ref] + [a for a in alt]
                GT = block[9].split(':')[0]
                genotype_row = []
                for geno_num in GT.split('/'):
                    genotype_row.append(table[int(geno_num)])

                # print(genotype_row, i)
                genotype[i] = ''.join(genotype_row)
        else:
            ref = per_unique_loc.split(':')[2]
            genotype[i] = ""+ref+ref

            
    
    genotype.append(single_vcf_file.name)

    return genotype

In [88]:
GT = dict()
def addtolist(l):
    for res in l:
        GT[res[-1]] = res[:-1]


In [89]:
p = multiprocessing.Pool(4)
p.map_async(getGenotype, all_path, callback=addtolist)
p.close()
p.join()

In [101]:
df = pd.DataFrame(GT)
df.index = shareset_list

In [105]:
str(df[1:2].index)

"Index(['chr1:10166:C:G'], dtype='object')"

In [114]:
a = df.iterrows()

In [117]:
a = next(a)

In [146]:
a[1]['small_2013NGS002B1_TSVC.vcf']

'CC'

In [156]:
from scipy import stats

def calfisher(row):
    
    index = row[0].split(":")
    
    chrom = index[0][3:]
    pos = index[1]
    
    ref = index[2]
    ref = ref+ref

    Con_AA = 0
    Con_AB = 0

    for con in controls:
        if row[1][con] == ref:
            Con_AA += 1
        else:
            Con_AB += 1

    Pat_AA = 0
    Pat_AB = 0
    for pat in patients:
        if row[1][pat] == ref:
            Pat_AA += 1
        else:
            Pat_AB += 1

    oddsratio, pvalue = stats.fisher_exact([[Con_AA, Pat_AA],[Con_AB,Pat_AB]])

    return [chrom, pos, pvalue]

In [163]:
result = list()

def addtoresult(l):
    for res in l:
        result.append(res)

In [164]:
p = multiprocessing.Pool(4)
p.map_async(calfisher, df.iterrows(), callback=addtoresult)
p.close()
p.join()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [167]:
pd.DataFrame(result, columns=['CHR','BP','P']).to_csv('./result.csv')

In [157]:
calfisher(next(df.iterrows()))

['1', '10162', 0.47368421052631615]

In [140]:
controls = [x.name for x in all_path[0:10] ]
patients = [x.name for x in all_path[10:]]

['small_2013NGS002B1_TSVC.vcf',
 'small_2013NGS005E1_TSVC.vcf',
 'small_2013NGS008H1_TSVC.vcf',
 'small_2013NGS010B2_TSVC.vcf',
 'small_2013NGS018B3_TSVC.vcf',
 'small_2013NGS020D3_TSVC.vcf',
 'small_2013NGS024H3_TSVC.vcf',
 'small_2013NGS035C5_TSVC.vcf',
 'small_2013NGS036D5_TSVC.vcf',
 'small_2013NGS042B6_TSVC.vcf']